<a href="https://colab.research.google.com/github/Ainuralin/public-transport/blob/main/bus_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas openpyxl folium

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving routes.xlsx to routes.xlsx
Saving bus-stops.xlsx to bus-stops.xlsx


In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [ ]:
routes_dict = pd.read_excel('routes.xlsx', sheet_name=None)
bus_stops_df = pd.read_excel('bus-stops.xlsx')

routes_df = pd.concat(
    [df.assign(sheet_name=sheet) for sheet, df in routes_dict.items()],
    ignore_index=True
)

print(routes_df.head())
print(bus_stops_df.head())

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


                               route_id   latitude  longitude     type  \
0  7bc7a718-79aa-4ee9-970b-9c0c67eee695  51.194895  71.408376  forward   
1  7bc7a718-79aa-4ee9-970b-9c0c67eee695  51.194995  71.408010  forward   
2  7bc7a718-79aa-4ee9-970b-9c0c67eee695  51.195009  71.407796  forward   
3  7bc7a718-79aa-4ee9-970b-9c0c67eee695  51.194977  71.407708  forward   
4  7bc7a718-79aa-4ee9-970b-9c0c67eee695  51.194919  71.407628  forward   

  sheet_name  
0         21  
1         21  
2         21  
3         21  
4         21  
                            bus_stop_id   latitude  longitude  \
0  438a47db-94e0-4571-887c-c2c52183cb43  51.117302  71.213309   
1  93d8b34a-8b16-4426-a186-511a0d9afeaa  51.100999  71.429194   
2  4783c9c3-f2b6-4c27-b13f-3133bb254e62  51.125393  71.432669   
3  4a0fd420-865a-4583-b140-26e534e28744  51.132029  71.284714   
4  bc0bd604-b6e6-40d1-8a79-fdab7059e9cf  51.281876  71.652801   

                  bus_stop_name  
0       Село Караоткел с. Акмол  
1     

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
bus_stops_df['latitude'] = pd.to_numeric(bus_stops_df['latitude'], errors='coerce')
bus_stops_df['longitude'] = pd.to_numeric(bus_stops_df['longitude'], errors='coerce')

original_len = len(bus_stops_df)

bus_stops_df = bus_stops_df.dropna(subset=['latitude', 'longitude'])

print("Было строк:", original_len)
print("Стало строк после удаления NaN:", len(bus_stops_df))

Было строк: 2094
Стало строк после удаления NaN: 2087


In [ ]:
bus_stops_df = bus_stops_df[bus_stops_df['latitude'] > 30]

In [ ]:
bus_stops_df = bus_stops_df[
    (bus_stops_df['latitude'] >= -90) & (bus_stops_df['latitude'] <= 90) &
    (bus_stops_df['longitude'] >= -180) & (bus_stops_df['longitude'] <= 180)
]

print(bus_stops_df[['latitude', 'longitude']].describe())

          latitude    longitude
count  2084.000000  2084.000000
mean     51.139629    71.425511
std       0.069927     0.146049
min      50.806694    70.919463
25%      51.113375    71.361814
50%      51.140526    71.418941
75%      51.182387    71.486399
max      51.473718    72.181557


### 1. Визуализация маршрутов и остановок на карте (Python)

In [ ]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
from scipy.spatial import cKDTree

bus_numbers = sorted(routes_df['sheet_name'].unique())

bus_dropdown = widgets.Dropdown(
    options=bus_numbers,
    description='Автобус:',
    layout={'width': '400px'}
)

show_button = widgets.Button(
    description='Показать маршрут',
    button_style='success'
)

map_output = widgets.Output()

def create_kdtree(route_coords):
    coords_array = np.array([(lon, lat) for lat, lon in route_coords])
    return cKDTree(coords_array)

def is_near_route_kdtree(stop_lat, stop_lon, kdtree, max_distance_km=0.03):
    max_distance_deg = max_distance_km / 111  # rough conversion
    stop_point = (stop_lon, stop_lat)
    dist, _ = kdtree.query(stop_point, distance_upper_bound=max_distance_deg)
    return dist != np.inf

def on_button_clicked(b):
    with map_output:
        clear_output()
        selected_bus = bus_dropdown.value
        bus_data = routes_df[routes_df['sheet_name'] == selected_bus]

        if bus_data.empty:
            print("Нет данных для выбранного автобуса.")
            return

        avg_lat = bus_data['latitude'].mean()
        avg_lon = bus_data['longitude'].mean()
        m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)

        route_coords = []

        for direction in ['forward', 'backward']:
            segment = bus_data[bus_data['type'] == direction]
            if not segment.empty:
                coords = list(zip(segment['latitude'], segment['longitude']))
                route_coords.extend(coords)
                color = 'green' if direction == 'forward' else 'blue'
                folium.PolyLine(
                    locations=coords,
                    color=color,
                    weight=5,
                    opacity=0.6,
                    tooltip=f"{selected_bus} ({direction})"
                ).add_to(m)

        if not route_coords:
            print("Нет координат маршрута.")
            return

        # KD-дерево для ускоренной проверки
        kdtree = create_kdtree(route_coords)

        for _, stop in bus_stops_df.iterrows():
            if is_near_route_kdtree(stop['latitude'], stop['longitude'], kdtree):
                folium.Marker(
                    location=(stop['latitude'], stop['longitude']),
                    popup=stop['bus_stop_name'],
                    icon=folium.Icon(color='red', icon='info-sign')
                ).add_to(m)

        display(m)

show_button.on_click(on_button_clicked)
display(widgets.VBox([bus_dropdown, show_button, map_output]))


In [ ]:
routes_df.groupby(['route_id', 'type']).size().unstack(fill_value=0).head(10)

type,backward,forward
route_id,,
0082483b-68ed-4060-8849-5db90b7a4ae9,96,120
0140f111-f6fe-467a-a62a-221ccd310042,303,298
058037b6-79a7-49c0-81e1-d5b36c317258,158,153
104c32ef-e3dc-47ee-8e24-31a54e257a0c,263,249
137c712d-f2fb-4edb-b253-bf1d6a8a27b5,131,137
1478f25d-c218-4adb-b3e1-71bb0bd66791,224,262
18721bf6-4a6b-4318-82a2-226271532a27,220,214
18fcb759-0dbf-4cb8-97a2-3ce199bf7f18,181,158
1aadf02c-b420-4c17-bb64-ec2149d95212,166,163


In [ ]:
print("Изначально:", routes_df['route_id'].nunique())

routes_df['latitude'] = pd.to_numeric(routes_df['latitude'], errors='coerce')
routes_df['longitude'] = pd.to_numeric(routes_df['longitude'], errors='coerce')

routes_df_clean = routes_df[
    (routes_df['latitude'].between(40, 60)) &
    (routes_df['longitude'].between(60, 80))
].dropna(subset=['latitude', 'longitude'])

print("После фильтрации:", routes_df_clean['route_id'].nunique())

Изначально: 114
После фильтрации: 114


### 2. Длина маршрута: сумма отрезков между остановками

In [ ]:
from scipy.spatial import cKDTree

route_coords = routes_df_clean[['latitude', 'longitude']].to_numpy()
stop_coords = bus_stops_df[['latitude', 'longitude']].to_numpy()

stop_tree = cKDTree(stop_coords)

distances, indices = stop_tree.query(route_coords, k=1)

threshold_degrees = 0.00045

mask = distances < threshold_degrees

matched_points = routes_df_clean[mask].copy()
matched_points['bus_stop_id'] = bus_stops_df.iloc[indices[mask]]['bus_stop_id'].values
matched_points['bus_stop_name'] = bus_stops_df.iloc[indices[mask]]['bus_stop_name'].values


In [ ]:
routes_ordered = matched_points.copy()
routes_ordered['point_index'] = routes_ordered.groupby(['route_id', 'type'])\
                                .cumcount()

ordered_stops = routes_ordered.sort_values(['route_id', 'type', 'point_index'])[
    ['route_id', 'type', 'bus_stop_id', 'bus_stop_name', 'latitude', 'longitude', 'point_index']
].drop_duplicates(['route_id', 'type', 'bus_stop_id'])  # Убираем дубли остановок

In [ ]:
routes_df = routes_df.copy()
routes_df['point_index'] = routes_df.groupby(['sheet_name', 'type']).cumcount()

In [ ]:
from geopy.distance import geodesic

segments = []

for (bus, direction), group in routes_df.groupby(['sheet_name', 'type']):
    group_sorted = group.sort_values('point_index').reset_index(drop=True)

    for i in range(len(group_sorted) - 1):
        stop_a = group_sorted.iloc[i]
        stop_b = group_sorted.iloc[i + 1]

        dist_km = geodesic(
            (stop_a['latitude'], stop_a['longitude']),
            (stop_b['latitude'], stop_b['longitude'])
        ).kilometers

        segments.append({
            'bus_number': bus,
            'direction': direction,
            'start_stop': stop_a['bus_stop_name'] if 'bus_stop_name' in stop_a else f"Point {stop_a['point_index']}",
            'end_stop': stop_b['bus_stop_name'] if 'bus_stop_name' in stop_b else f"Point {stop_b['point_index']}",
            'segment_length_km': round(dist_km, 3)
        })

segments_df = pd.DataFrame(segments)


In [ ]:
total_lengths = segments_df.groupby(['bus_number', 'direction'])['segment_length_km'].sum().unstack()

total_lengths = total_lengths.rename(columns={
    'forward': 'forward_km',
    'backward': 'backward_km'
}).fillna(0)

total_lengths['total_km'] = total_lengths['forward_km'] + total_lengths['backward_km']

def classify(length):
    if length < 10:
        return 'менее 10 км'
    elif length < 25:
        return 'от 10 до 25 км'
    elif length < 35:
        return 'от 25 до 35 км'
    elif length < 50:
        return 'от 35 до 50 км'
    else:
        return 'более 50 км'

total_lengths['category'] = total_lengths['total_km'].apply(classify)

total_lengths.reset_index(inplace=True)
total_lengths.head()

direction,bus_number,backward_km,forward_km,total_km,category
0,1,12.765,11.288,24.053,от 10 до 25 км
1,10,30.891,26.167,57.058,более 50 км
2,11,23.664,24.215,47.879,от 35 до 50 км
3,12,24.138,23.710,47.848,от 35 до 50 км
4,120,18.905,17.727,36.632,от 35 до 50 км


In [ ]:
category_table = total_lengths.groupby('category').agg({
    'bus_number': lambda x: list(x),
    'total_km': 'count'
}).rename(columns={
    'bus_number': 'routes',
    'total_km': 'number_of_routes'
}).reset_index()

category_order = ['менее 10 км', 'от 10 до 25 км', 'от 25 до 35 км', 'от 35 до 50 км', 'более 50 км']
category_table['category'] = pd.Categorical(category_table['category'], categories=category_order, ordered=True)

category_table = category_table.sort_values('category')

category_table


direction,category,routes,number_of_routes
1,от 10 до 25 км,"[1, 121, 25, 55А, 55Б]",5
2,от 25 до 35 км,"[13, 16, 23, 314, 39, 44, 504, 57, 6, 69 будни...",11
3,от 35 до 50 км,"[11, 12, 120, 120А, 14, 17, 19, 20, 21, 22, 24...",55
0,более 50 км,"[10, 15, 15А, 18, 2, 22А, 24Н, 28, 29А, 302, 3...",43


In [ ]:
category_table.to_excel("route_length_categories.xlsx", index=False)

### 3. Среднее и медианное расстояние между остановками по маршрутам

In [ ]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

# Distance between points
def haversine(coord1, coord2):
    lon1, lat1, lon2, lat2 = map(radians, [coord1[1], coord1[0], coord2[1], coord2[0]])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # The radius of the Earth in km
    return c * r

def find_nearest_stop(point, bus_stops_df):
    min_dist = float('inf')
    nearest_name = None
    for _, stop in bus_stops_df.iterrows():
        dist = haversine(point, (stop['latitude'], stop['longitude']))
        if dist < min_dist:
            min_dist = dist
            nearest_name = stop['bus_stop_name']
    return nearest_name

results = []

for bus_number in routes_df['sheet_name'].unique():
    bus_data = routes_df[routes_df['sheet_name'] == bus_number]

    route_info = {
        'Номер': bus_number,
        'Начальная остановка': None,
        'Конечная остановка': None,
        'Длина оборота (км.)': 0,
        'Среднее расстояние (прямое) (м.)': None,
        'Медианное расстояние (прямое) (м.)': None,
        'Среднее расстояние (обратное) (м.)': None,
        'Медианное расстояние (обратное) (м.)': None
    }

    total_length_km = 0

    for direction in ['forward', 'backward']:
        segment = bus_data[bus_data['type'] == direction]
        if segment.empty or len(segment) < 2:
            continue

        coords = list(zip(segment['latitude'], segment['longitude']))
        distances = [haversine(coords[i], coords[i+1]) * 1000 for i in range(len(coords)-1)]
        total_length_km += sum(distances) / 1000

        mean_dist = np.mean(distances)
        median_dist = np.median(distances)

        if direction == 'forward':
            route_info['Среднее расстояние (прямое) (м.)'] = round(mean_dist, 2)
            route_info['Медианное расстояние (прямое) (м.)'] = round(median_dist, 2)
            route_info['Начальная остановка'] = find_nearest_stop(coords[0], bus_stops_df)
            route_info['Конечная остановка'] = find_nearest_stop(coords[-1], bus_stops_df)
        else:
            route_info['Среднее расстояние (обратное) (м.)'] = round(mean_dist, 2)
            route_info['Медианное расстояние (обратное) (м.)'] = round(median_dist, 2)

    route_info['Длина оборота (км.)'] = round(total_length_km, 2)
    results.append(route_info)

summary_df = pd.DataFrame(results)

def categorize_distance(dist):
    if pd.isna(dist):
        return 'нет данных'
    elif dist < 300:
        return 'до 300 м'
    elif dist < 500:
        return 'от 300 до 500 м'
    else:
        return 'больше 500 м'

summary_df['Медиана общая'] = summary_df[[
    'Медианное расстояние (прямое) (м.)',
    'Медианное расстояние (обратное) (м.)'
]].mean(axis=1)

summary_df['Категория маршрута'] = summary_df['Медиана общая'].apply(categorize_distance)


summary_df = summary_df[[
    'Номер',
    'Начальная остановка',
    'Конечная остановка',
    'Длина оборота (км.)',
    'Среднее расстояние (прямое) (м.)',
    'Медианное расстояние (прямое) (м.)',
    'Среднее расстояние (обратное) (м.)',
    'Медианное расстояние (обратное) (м.)',
    'Категория маршрута'
]]

# Показать результат
display(summary_df)

,Номер,Начальная остановка,Конечная остановка,Длина оборота (км.),Среднее расстояние (прямое) (м.),Медианное расстояние (прямое) (м.),Среднее расстояние (обратное) (м.),Медианное расстояние (обратное) (м.),Категория маршрута
0,21,Железнодорожный вокзал конечный пункт,Ж/д вокзал Нурлы жол конечный пункт,46.09,104.53,61.75,99.48,62.44,до 300 м
1,73,ЖК Коктем ул. Кумисбекова,Жилой массив «Промышленный» конечный пункт,47.48,85.90,52.34,95.47,63.60,до 300 м
2,302,Кардиохирургическая клиника пос. Каркаралы,Село Тайтобе конечный пункт,53.19,152.62,84.85,116.10,55.86,до 300 м
3,10,Железнодорожный вокзал конечный пункт,Международный аэропорт конечный пункт,56.98,110.26,50.13,121.94,49.79,до 300 м
4,9,Железнодорожный вокзал конечный пункт,Детский сад Даурен Ул. Коксай,44.67,110.55,77.03,113.49,90.75,до 300 м
...,...,...,...,...,...,...,...,...,...
109,120А,Жасыл Ел пр. Сарыарка,Учреждение ЕЦ 166/10 ул. Пушкина,36.00,154.83,80.05,159.78,93.43,до 300 м
110,5А,Гипермаркет «Строймарт» пр. Кошкарбаева,Театр танца «Наз» ул. Дулата Бабатайулы,46.19,105.56,75.41,103.36,75.41,до 300 м
111,3,Железнодорожный вокзал конечный пункт оставить,Ж/д вокзал Нурлы жол конечный пункт,43.61,94.32,48.30,96.18,51.16,до 300 м
112,59,Садоводческое общество «Ивушка» пос. Софиевка,Торговый дом «Көктем» пос. Талапкер,65.74,250.13,140.86,254.16,104.48,до 300 м


In [ ]:
print("Минимальное и максимальное значение для каждого столбца:\n")

for column in [
    'Длина оборота (км.)',
    'Среднее расстояние (прямое) (м.)',
    'Медианное расстояние (прямое) (м.)',
    'Среднее расстояние (обратное) (м.)',
    'Медианное расстояние (обратное) (м.)'
]:
    min_val = summary_df[column].min()
    max_val = summary_df[column].max()
    print(f"{column}: мин = {min_val}, макс = {max_val}")


Минимальное и максимальное значение для каждого столбца:

Длина оборота (км.): мин = 23.1, макс = 181.38
Среднее расстояние (прямое) (м.): мин = 69.3, макс = 286.1
Медианное расстояние (прямое) (м.): мин = 37.07, макс = 140.86
Среднее расстояние (обратное) (м.): мин = 29.71, макс = 334.27
Медианное расстояние (обратное) (м.): мин = 29.71, макс = 129.56


In [ ]:
# Маршрут с максимальной длиной оборота
max_len_row = summary_df.loc[summary_df['Длина оборота (км.)'].idxmax()]
print("\nМаршрут с максимальной длиной оборота:")
print(max_len_row)

# Маршрут с минимальным средним расстоянием (прямое)
min_avg_row = summary_df.loc[summary_df['Среднее расстояние (прямое) (м.)'].idxmin()]
print("\nМаршрут с минимальным средним расстоянием (прямое):")
print(min_avg_row)



Маршрут с максимальной длиной оборота:
Номер                                                                                 315
Начальная остановка                     Акмолинская областная больница № 2 пр. Абылай ...
Конечная остановка                                            Село Ақбулақ конечный пункт
Длина оборота (км.)                                                                181.38
Среднее расстояние (прямое) (м.)                                                   231.97
Медианное расстояние (прямое) (м.)                                                  96.99
Среднее расстояние (обратное) (м.)                                                 273.14
Медианное расстояние (обратное) (м.)                                                 90.7
Категория маршрута                                                               до 300 м
Name: 77, dtype: object

Маршрут с минимальным средним расстоянием (прямое):
Номер                                                                    

### 4. Дублируемость маршрутов

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.neighbors import BallTree
import numpy as np

# Подготовим координаты в радианах
stop_coords_rad = np.radians(bus_stops_df[['latitude', 'longitude']].values)
stop_names = bus_stops_df['bus_stop_name'].values
tree = BallTree(stop_coords_rad, metric='haversine')

radius = 0.03 / 6371.0

route_stops_dict = {}

for bus_number in routes_df['sheet_name'].unique():
    bus_data = routes_df[routes_df['sheet_name'] == bus_number]
    coords_rad = np.radians(bus_data[['latitude', 'longitude']].values)

    indices = tree.query_radius(coords_rad, r=radius)
    stops = set()
    for ind_list in indices:
        for idx in ind_list:
            stops.add(stop_names[idx])

    route_stops_dict[bus_number] = stops

In [ ]:
from itertools import combinations
import pandas as pd

duplicate_results = []

for route1, route2 in combinations(route_stops_dict.keys(), 2):
    stops1 = route_stops_dict[route1]
    stops2 = route_stops_dict[route2]

    if not stops1 or not stops2:
        continue

    intersection = stops1 & stops2
    count_common = len(intersection)
    percent1 = round(100 * count_common / len(stops1), 2)
    percent2 = round(100 * count_common / len(stops2), 2)

    is_duplicate = percent1 >= 50 or percent2 >= 50

    duplicate_results.append({
        'Маршрут 1': route1,
        'Маршрут 2': route2,
        'Совпадающих остановок': count_common,
        '% совпад. к маршруту 1': percent1,
        '% совпад. к маршруту 2': percent2,
        'Дублируются (да/нет)': 'да' if is_duplicate else 'нет'
    })

duplicates_df = pd.DataFrame(duplicate_results)
display(duplicates_df)


,Маршрут 1,Маршрут 2,Совпадающих остановок,% совпад. к маршруту 1,% совпад. к маршруту 2,Дублируются (да/нет)
0,21,73,4,4.88,4.26,нет
1,21,302,0,0.00,0.00,нет
2,21,10,10,12.20,13.51,нет
3,21,9,5,6.10,5.75,нет
4,21,25,17,20.73,34.00,нет
...,...,...,...,...,...,...
6436,5А,59,0,0.00,0.00,нет
6437,5А,319,0,0.00,0.00,нет
6438,3,59,0,0.00,0.00,нет
6439,3,319,0,0.00,0.00,нет


In [ ]:
duplicated_routes = set(duplicates_df[duplicates_df['Дублируются (да/нет)'] == 'да']['Маршрут 1']) | \
                    set(duplicates_df[duplicates_df['Дублируются (да/нет)'] == 'да']['Маршрут 2'])

print(f"Количество маршрутов, имеющих дубликаты: {len(duplicated_routes)} из {routes_df['sheet_name'].nunique()}")


Количество маршрутов, имеющих дубликаты: 56 из 114


In [ ]:
from collections import Counter

counter = Counter(duplicates_df[duplicates_df['Дублируются (да/нет)'] == 'да']['Маршрут 1'].tolist() +
                  duplicates_df[duplicates_df['Дублируются (да/нет)'] == 'да']['Маршрут 2'].tolist())

top_10 = counter.most_common(10)
print("Топ-10 маршрутов по числу дубликатов:")
for route, count in top_10:
    print(f"Маршрут {route}: {count} дубликатов")

Топ-10 маршрутов по числу дубликатов:
Маршрут 321: 8 дубликатов
Маршрут 327: 8 дубликатов
Маршрут 316: 8 дубликатов
Маршрут 313: 8 дубликатов
Маршрут 318: 8 дубликатов
Маршрут 304: 7 дубликатов
Маршрут 328: 7 дубликатов
Маршрут 315: 7 дубликатов
Маршрут 322: 7 дубликатов
Маршрут 311: 6 дубликатов


### 5. Пересадочные узлы

In [ ]:
import numpy as np
from collections import defaultdict

stop_to_routes = defaultdict(set)

for bus_number in routes_df['sheet_name'].unique():
    bus_data = routes_df[routes_df['sheet_name'] == bus_number]
    route_coords_rad = np.radians(bus_data[['latitude', 'longitude']].values)

    indices = tree.query_radius(route_coords_rad, r=radius)

    for neighbors in indices:
        for idx in neighbors:
            stop_name = stop_names[idx]
            stop_to_routes[stop_name].add(bus_number)

results = [{
    'Остановка': stop,
    'Количество маршрутов': len(routes),
    'Маршруты, проходящие через остановку': sorted(routes)
} for stop, routes in stop_to_routes.items()]

hubs_df = pd.DataFrame(results).sort_values(by='Количество маршрутов', ascending=False)
display(hubs_df.head(10))


,Остановка,Количество маршрутов,"Маршруты, проходящие через остановку"
120,Национальный научный медицинский центр пр. Тау...,27,"[11, 13, 15, 15А, 22, 22А, 28, 304, 31, 313, 3..."
144,Национальный научный медицинский центр Ул. Кен...,25,"[11, 15, 15А, 22, 22А, 28, 304, 31, 313, 315, ..."
118,Торговый дом «Гүлжан» ж/м Куйгенжар,25,"[11, 13, 22, 22А, 28, 304, 31, 313, 315, 316, ..."
145,Международная школа «Мирас» Ул. Кенесары,23,"[11, 13, 22, 22А, 28, 304, 31, 313, 315, 316, ..."
0,Железнодорожный вокзал конечный пункт,22,"[10, 12, 14, 19, 21, 22, 22А, 23, 25, 26, 3, 3..."
451,Торговый центр «Әлем» пос. Талапкер,22,"[120, 120А, 13, 16, 17, 22, 22А, 24, 24Н, 307,..."
81,Железнодорожный вокзал конечный пункт оставить,22,"[10, 12, 14, 19, 21, 22, 22А, 23, 25, 26, 3, 3..."
119,Международная школа «Мирас» ж/м Куйгенжар,21,"[11, 22, 22А, 304, 31, 313, 315, 316, 318, 321..."
27,Дом министерств пр. Кабанбай батыра,21,"[12, 15, 15А, 18, 19, 21, 28, 30, 35, 40, 46, ..."
117,Торговый дом «Гүлжан» Ул. Кенесары,20,"[11, 13, 22, 22А, 23, 28, 31, 318, 33, 48, 5, ..."


### 6. Зигзагообразные и кольцевые (петлеобразные) маршруты

In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from math import degrees
import folium

def angle_between_points(p1, p2, p3):
    a, b, c = np.array(p1), np.array(p2), np.array(p3)
    ba = a - b
    bc = c - b
    norm_ba = np.linalg.norm(ba)
    norm_bc = np.linalg.norm(bc)
    if norm_ba == 0 or norm_bc == 0:
        return 180
    cosine_angle = np.dot(ba, bc) / (norm_ba * norm_bc)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return degrees(angle)

def classify_bus_routes(df, loop_threshold_km=0.5, zigzag_angle=90, zigzag_count_threshold=5):
    route_shapes = []

    for (route_id, direction), group in df.groupby(['sheet_name', 'type']):
        group_sorted = group.sort_values(by='point_index').reset_index(drop=True)
        coords = list(zip(group_sorted['latitude'], group_sorted['longitude']))

        if len(coords) < 3:
            continue

        # Проверка на кольцевой маршрут
        start_point, end_point = coords[0], coords[-1]
        loop_distance = geodesic(start_point, end_point).km
        is_loop = loop_distance <= loop_threshold_km

        # Подсчёт резких поворотов
        sharp_turns = 0
        for i in range(1, len(coords) - 1):
            angle = angle_between_points(coords[i - 1], coords[i], coords[i + 1])
            if angle < zigzag_angle:
                sharp_turns += 1

        if is_loop:
            route_type = 'кольцевой'
            reason = f'Начало и конец маршрута находятся на расстоянии {round(loop_distance * 1000)} м.'
        elif sharp_turns >= zigzag_count_threshold:
            route_type = 'зигзагообразный'
            reason = f'Маршрут имеет {sharp_turns} поворотов с углом менее {zigzag_angle}°.'
        else:
            route_type = 'линейный'
            reason = f'Маршрут имеет {sharp_turns} острых поворотов и не является кольцевым.'

        route_shapes.append({
            'Маршрут': route_id,
            'Направление': direction,
            'Тип маршрута': route_type,
            'Комментарий': reason
        })

    return pd.DataFrame(route_shapes)

route_shapes_df = classify_bus_routes(routes_df)
display(route_shapes_df)


,Маршрут,Направление,Тип маршрута,Комментарий
0,1,backward,линейный,Маршрут имеет 4 острых поворотов и не является...
1,1,forward,линейный,Маршрут имеет 4 острых поворотов и не является...
2,10,backward,зигзагообразный,Маршрут имеет 12 поворотов с углом менее 90°.
3,10,forward,зигзагообразный,Маршрут имеет 8 поворотов с углом менее 90°.
4,11,backward,зигзагообразный,Маршрут имеет 12 поворотов с углом менее 90°.
...,...,...,...,...
221,84,forward,зигзагообразный,Маршрут имеет 10 поворотов с углом менее 90°.
222,85,backward,зигзагообразный,Маршрут имеет 5 поворотов с углом менее 90°.
223,85,forward,зигзагообразный,Маршрут имеет 10 поворотов с углом менее 90°.
224,9,backward,зигзагообразный,Маршрут имеет 7 поворотов с углом менее 90°.
